In [28]:
import pandas as pd
import math
import re
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [30]:
#load data
country_names = pd.read_csv("../Section_3_4/country_metadata/un_countries_meta.csv")
#Old version of this code included countries, Kosovo, Palestine, and Taiwan which were not included in the final analysis
keywords = country_names['Name'].unique()

In [31]:
def write_to_file(result, output_file_name):
    result.columns = ['keyword', 'sentence']
    result.to_csv(output_file_name, mode='a', index=False, header=False)
    return pd.DataFrame()

In [ ]:
f = open("pre_training_country_text.txt")

result = pd.DataFrame()
text = []
for line in f:
    for sentence in tokenizer.tokenize(line):
         for keyword in keywords:
            if (keyword in sentence) and (re.search(r"\b" + keyword + r"\b", sentence) is not None):
                result = result.append([[keyword, sentence]])
    #clear cache
    if (len(result) > 5000):    
        result = write_to_file(result, 'pre_training_country_text.csv')
result

In [33]:
data = pd.read_csv("pre_training_country_text.csv", header=None)

In [34]:
data.columns = ['country', 'text']
#remove all entries that have more than one country in them
data = data.drop_duplicates(subset=['text'])
#keep only one word countries
data = data[~data['country'].str.contains(" ")]
data = data.dropna()
data = data[~data['text'].str.contains("doc id")]

,country,text
0,Iran,"Arif ( also spelled Aref in Pushtu, Persian & ..."
121,Japan,The Communist Party of Japan was instructed to...
122,Netherlands,"By the time Marius was six years old, his fami..."
123,Estonia,Because Sweden placed 19th at the 2005 contest...
126,Sweden,Meaning that Sweden automatically qualified fo...
...,...,...
881202,Angola,"The Portuguese Empire, then LHT, introduced LH..."
881204,Portugal,Although Portugal itself switched to RHT in 19...
881206,Angola,Other former Portuguese colonies in Africa inc...
881209,Nauru,"In May 2016, the Parliament of Nauru passed th..."


In [36]:
import random
#interested_countries = random.sample(set(data['country'].unique()), 20)
#saved for reproducibility
interested_countries = ['Syria','Serbia','Myanmar','Uzbekistan','Chad','Egypt','Qatar','Madagascar','Angola','Jamaica','Barbados','Guinea','Cambodia','Iceland','Iran','Bahamas','Senegal','Uganda','Poland','India']

In [37]:
fine_tuning_data = data[data['country'].isin(interested_countries)]
fine_tuning_data = fine_tuning_data.reset_index(drop=True)
fine_tuning_data['text'] = fine_tuning_data['text'].apply(lambda x: x + " \n")


In [38]:
training_data = fine_tuning_data.groupby("country").apply(lambda x: x.sample(n=1000)).reset_index(drop=True)

In [39]:
test_data = fine_tuning_data[~fine_tuning_data.index.isin(training_data.index)]

In [40]:
len(fine_tuning_data), len(training_data), len(test_data)

(41487, 20000, 21487)

In [41]:
#Unfortunately, the original data for this got overwritten, but this is the code for how it was originally created
training_data[['text']].to_csv("fine_tuning_training_data.csv", index=False)
test_data[['text']].to_csv("fine_tuning_test_data.csv", index=False)